Loading data

In [13]:
import pandas as pd
import numpy as np
from scipy import stats

df_inception_old = pd.read_csv (r'C:\Users\20192236\Downloads\results\results\Inception_submission.csv')
df_efficient_old = pd.read_csv (r'C:\Users\20192236\Downloads\results\results\EfficientNet_submission.csv')
df_resnet_old = pd.read_csv (r'C:\Users\20192236\Downloads\results\results\ResNet_submission.csv')
df_vgg16_old = pd.read_csv (r'C:\Users\20192236\Downloads\results\results\VGG16_submission.csv')
df_xception_old = pd.read_csv (r'C:\Users\20192236\Downloads\results\results\Xception_submission.csv')

df_inception = df_inception_old.rename(columns={'label':'label_inception'}, errors='raise')
df_efficient = df_efficient_old.rename(columns={'label':'label_efficient'}, errors='raise')
df_resnet = df_resnet_old.rename(columns={'label':'label_resnet'}, errors='raise')
df_vgg16 = df_vgg16_old.rename(columns={'label':'label_vgg16'}, errors='raise')
df_xception = df_xception_old.rename(columns={'label':'label_xception'}, errors='raise')

all_probs = pd.DataFrame(data=[df_inception['id'], df_inception['label_inception'], df_xception['label_xception'], df_efficient['label_efficient'], df_resnet['label_resnet'], df_vgg16['label_vgg16']])
all_probs = all_probs.transpose()

,id,label_inception,label_xception,label_efficient,label_resnet,label_vgg16
0,00006537328c33e284c973d7b39d340809f7271b,0.478034,0.993316,0.503348,0.999171,0.998745
1,0000ec92553fda4ce39889f9226ace43cae3364e,0.518892,0.95245,0.959156,0.93481,0.6989
2,00024a6dee61f12f7856b0fc6be20bc7a48ba3d2,0.575155,0.835915,0.396475,0.992789,0.89388
3,000253dfaa0be9d0d100283b22284ab2f6b643f6,0.648605,0.675391,0.560131,0.984799,0.996066
4,000270442cc15af719583a8172c87cd2bd9c7746,0.192873,0.002332,0.019242,0.001395,0.01143
...,...,...,...,...,...,...
57453,fffdd1cbb1ac0800f65309f344dd15e9331e1c53,0.147148,0.000633,0.022752,0.035865,0.001352
57454,fffdf4b82ba01f9cae88b9fa45be103344d9f6e3,0.176237,0.013224,0.001911,0.048932,0.009463
57455,fffec7da56b54258038b0d382b3d55010eceb9d7,0.290546,0.000219,0.014254,0.0145,0.001723
57456,ffff276d06a9e3fffc456f2a5a7a3fd1a2d322c6,0.420927,0.503107,0.0917,0.757315,0.834874


Making predictions

In [14]:
mu = []
sigma = []
for i in range(len(all_probs)):
    mu.append(np.mean(all_probs.iloc[i, 1:6].values))
    sigma.append(np.std(all_probs.iloc[i, 1:6].values))

In [29]:
def predict_Bayes_class(X,mu_list,sigma_list): 
    #Returns the predicted class from an optimal bayes classifier - distributions must be known
    scores_list = []
    classes = len(mu_list)
    
    for p in range(classes):
        score = stats.multivariate_normal.pdf(X, mean=mu_list[p], cov=sigma_list[p])
        scores_list.append(score)
             
    return np.argmax(scores_list)

In [30]:
pred = []
for i in range(len(all_probs)):
        pred.append(predict_Bayes_class(all_probs.iloc[i, 1:6].values, [mu[i]], [sigma[i]]))

In [62]:
proba_list = []
proba_list_round = []
for i in range(len(pred)):
    if pred[i] == 0:
        proba_list.append(all_probs.iloc[i, 1])
    elif pred[i] == 1:
        proba_list.append(all_probs.iloc[i, 2])
    elif pred[i] == 2:
        proba_list.append(all_probs.iloc[i, 3])
    elif pred[i] == 3:
        proba_list.append(all_probs.iloc[i, 4])
    elif pred[i] == 4:
        proba_list.append(all_probs.iloc[i, 5])
    proba_list_round.append(int(round(proba_list[i], 0)))
proba_list_round

df = pd.DataFrame(proba_list_round)
df = df.set_index(all_probs['id'])
df = df.rename(columns={0: "label"})

df.to_csv(r"C:\Users\20192236\Documents\Project_Imaging\BOC.csv")

1


,label
id,
00006537328c33e284c973d7b39d340809f7271b,1
0000ec92553fda4ce39889f9226ace43cae3364e,1
00024a6dee61f12f7856b0fc6be20bc7a48ba3d2,1
000253dfaa0be9d0d100283b22284ab2f6b643f6,1
000270442cc15af719583a8172c87cd2bd9c7746,0
...,...
fffdd1cbb1ac0800f65309f344dd15e9331e1c53,0
fffdf4b82ba01f9cae88b9fa45be103344d9f6e3,0
fffec7da56b54258038b0d382b3d55010eceb9d7,0


The second difficulty in BOC is choosing the prior probability for each hypothesis p(hi). For simplicity, the prior is usually set to be the uniform distribution [30].